# Content-Based Movie Recommendation System

#### Recommendation systems are a collection of algorithms used to recommend items to users based on information taken from the user. These systems have become ubiquitous, and can be commonly seen in online stores, movies databases and job finders. In this notebook, we will explore Content-based recommendation systems and implement a simple version of one using Python and the Pandas library.


In [1]:
import pandas as pd
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
movies = pd.read_csv('C:/Users/Adhish/Documents/Jupyter Notebook/Data/ml-25m.zip/ml-latest/movies.csv')
movies.head()

movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy

## Preprocessing

Extract Year from the **Title** column and store it in a new **Year** Column.

In [3]:
movies['year'] = movies.title.str.extract( '(\(\d{4}\))', expand=False)
movies.head()

movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres    year  
0  Adventure|Animation|Children|Comedy|Fantasy  (1995)  
1                   Adventure|Children|Fantasy  (1995)  
2                               Comedy|Romance  (1995)  
3                         Comedy|Drama|Romance  (1995)  
4                                       Comedy  (1995)

In [4]:
movies['year'] = movies.year.str.extract( '(\d{4})', expand=False)
movies.head()

movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  year  
0  Adventure|Animation|Children|Comedy|Fantasy  1995  
1                   Adventure|Children|Fantasy  1995  
2                               Comedy|Romance  1995  
3                         Comedy|Drama|Romance  1995  
4                                       Comedy  1995

In [5]:
movies['title'] = movies.title.str.replace('(\(\d{4}\))', '')
movies.head()

movieId                         title  \
0        1                    Toy Story    
1        2                      Jumanji    
2        3             Grumpier Old Men    
3        4            Waiting to Exhale    
4        5  Father of the Bride Part II    

                                        genres  year  
0  Adventure|Animation|Children|Comedy|Fantasy  1995  
1                   Adventure|Children|Fantasy  1995  
2                               Comedy|Romance  1995  
3                         Comedy|Drama|Romance  1995  
4                                       Comedy  1995

In [6]:
movies['title'] = movies.title.str.strip() # To rmevoe any whitespace characters
movies.head()

movieId                        title  \
0        1                    Toy Story   
1        2                      Jumanji   
2        3             Grumpier Old Men   
3        4            Waiting to Exhale   
4        5  Father of the Bride Part II   

                                        genres  year  
0  Adventure|Animation|Children|Comedy|Fantasy  1995  
1                   Adventure|Children|Fantasy  1995  
2                               Comedy|Romance  1995  
3                         Comedy|Drama|Romance  1995  
4                                       Comedy  1995

Let's also split the values in the **Genres** column into a **list of Genres** to simplify for future use. This can be achieved by applying Python's split string function on the correct column.


In [7]:
movies['genres'] = movies.genres.str.split('|')
movies.head()

movieId                        title  \
0        1                    Toy Story   
1        2                      Jumanji   
2        3             Grumpier Old Men   
3        4            Waiting to Exhale   
4        5  Father of the Bride Part II   

                                              genres  year  
0  [Adventure, Animation, Children, Comedy, Fantasy]  1995  
1                     [Adventure, Children, Fantasy]  1995  
2                                  [Comedy, Romance]  1995  
3                           [Comedy, Drama, Romance]  1995  
4                                           [Comedy]  1995

In [8]:
movies.shape

(34208, 4)

Since keeping genres in a list format isn't optimal, we will use the One Hot Encoding technique to convert the list of genres to a vector where each column corresponds to one possible value of the feature. In this case, we store every different genre in columns that contain either 1 or 0. 1 shows that a movie has that genre and 0 shows that it doesn't. Let's also store this dataframe in another variable since genres won't be important for our first recommendation system.


In [9]:
movies_genres = movies.copy()

In [10]:
for index, row in movies.iterrows():
    for genre in row['genres']:
        movies_genres.at[index, genre] = 1

In [11]:
movies_genres.head()

movieId                        title  \
0        1                    Toy Story   
1        2                      Jumanji   
2        3             Grumpier Old Men   
3        4            Waiting to Exhale   
4        5  Father of the Bride Part II   

                                              genres  year  Adventure  \
0  [Adventure, Animation, Children, Comedy, Fantasy]  1995        1.0   
1                     [Adventure, Children, Fantasy]  1995        1.0   
2                                  [Comedy, Romance]  1995        NaN   
3                           [Comedy, Drama, Romance]  1995        NaN   
4                                           [Comedy]  1995        NaN   

   Animation  Children  Comedy  Fantasy  Romance  ...  Horror  Mystery  \
0        1.0       1.0     1.0      1.0      NaN  ...     NaN      NaN   
1        NaN       1.0     NaN      1.0      NaN  ...     NaN      NaN   
2        NaN       NaN     1.0      NaN      1.0  ...     NaN      NaN   
3        NaN       NaN     1.0      NaN      1.0  ...     NaN      NaN   
4        NaN       NaN     1.0      NaN      NaN  ...     NaN      NaN   

   Sci-Fi  IMAX  Documentary  War  Musical  Western  Film-Noir  \
0     NaN   NaN          NaN  NaN      NaN      NaN        NaN   
1     NaN   NaN          NaN  NaN      NaN      NaN        NaN   
2     NaN   NaN          NaN  NaN      NaN      NaN        NaN   
3     NaN   NaN          NaN  NaN      NaN      NaN        NaN   
4     NaN   NaN          NaN  NaN      NaN      NaN        NaN   

   (no genres listed)  
0                 NaN  
1                 NaN  
2                 NaN  
3                 NaN  
4                 NaN  

[5 rows x 24 columns]

In [12]:
movies_genres = movies_genres.fillna(0)
movies_genres.head()

movieId                        title  \
0        1                    Toy Story   
1        2                      Jumanji   
2        3             Grumpier Old Men   
3        4            Waiting to Exhale   
4        5  Father of the Bride Part II   

                                              genres  year  Adventure  \
0  [Adventure, Animation, Children, Comedy, Fantasy]  1995        1.0   
1                     [Adventure, Children, Fantasy]  1995        1.0   
2                                  [Comedy, Romance]  1995        0.0   
3                           [Comedy, Drama, Romance]  1995        0.0   
4                                           [Comedy]  1995        0.0   

   Animation  Children  Comedy  Fantasy  Romance  ...  Horror  Mystery  \
0        1.0       1.0     1.0      1.0      0.0  ...     0.0      0.0   
1        0.0       1.0     0.0      1.0      0.0  ...     0.0      0.0   
2        0.0       0.0     1.0      0.0      1.0  ...     0.0      0.0   
3        0.0       0.0     1.0      0.0      1.0  ...     0.0      0.0   
4        0.0       0.0     1.0      0.0      0.0  ...     0.0      0.0   

   Sci-Fi  IMAX  Documentary  War  Musical  Western  Film-Noir  \
0     0.0   0.0          0.0  0.0      0.0      0.0        0.0   
1     0.0   0.0          0.0  0.0      0.0      0.0        0.0   
2     0.0   0.0          0.0  0.0      0.0      0.0        0.0   
3     0.0   0.0          0.0  0.0      0.0      0.0        0.0   
4     0.0   0.0          0.0  0.0      0.0      0.0        0.0   

   (no genres listed)  
0                 0.0  
1                 0.0  
2                 0.0  
3                 0.0  
4                 0.0  

[5 rows x 24 columns]

In [13]:
movies_genres.shape

(34208, 24)

Next, let's look at the Ratings.


In [14]:
ratings = pd.read_csv('C:/Users/Adhish/Documents/Jupyter Notebook/Data/ml-25m.zip/ml-latest/ratings.csv')

In [15]:
ratings.head()

userId  movieId  rating   timestamp
0       1      169     2.5  1204927694
1       1     2471     3.0  1204927438
2       1    48516     5.0  1204927435
3       2     2571     3.5  1436165433
4       2   109487     4.0  1436165496

Every row in the ratings dataframe has a user id associated with at least one movie, a rating and a timestamp showing when they reviewed it. We won't be needing the timestamp column, so let's drop it to save memory.


In [16]:
ratings = ratings.drop('timestamp', axis=1)
ratings.head()

userId  movieId  rating
0       1      169     2.5
1       1     2471     3.0
2       1    48516     5.0
3       2     2571     3.5
4       2   109487     4.0

# Content-Based Recommendation System

**Content-Based** or **Item-Item recommendation systems** attempts to figure out what a users favourite aspects of an item is, and then recommends items that present those aspects. In our case, we're going to try to figure out the input's favorite genres from the movies and ratings given.

Let's begin by creating an input user to recommend movies to:

Notice: To add more movies, simply increase the amount of elements in the **userInput**. Feel free to add more in! Just be sure to write it in with capital letters and if a movie starts with a "The", like "The Matrix" then write it in like this: 'Matrix, The' .


In [17]:
userinput = [
    
    {'title':'Breakfast Club, The', 'rating':5},
    {'title':'Toy Story', 'rating':3.5},
    {'title':'Jumanji', 'rating':2},
    {'title':'Pulp Fiction', 'rating':5},
    {'title':'Akira', 'rating':4.5}
    
 ]

In [18]:
userinput = pd.DataFrame(userinput)

In [19]:
userinput.head()

title  rating
0  Breakfast Club, The     5.0
1            Toy Story     3.5
2              Jumanji     2.0
3         Pulp Fiction     5.0
4                Akira     4.5

**Add MovieID to userinput.**

In [20]:
inputid = movies[movies['title'].isin(userinput.title.to_list())]
inputid

movieId                title  \
0           1            Toy Story   
1           2              Jumanji   
293       296         Pulp Fiction   
1246     1274                Akira   
1885     1968  Breakfast Club, The   

                                                 genres  year  
0     [Adventure, Animation, Children, Comedy, Fantasy]  1995  
1                        [Adventure, Children, Fantasy]  1995  
293                    [Comedy, Crime, Drama, Thriller]  1994  
1246             [Action, Adventure, Animation, Sci-Fi]  1988  
1885                                    [Comedy, Drama]  1985

In [21]:
input_id = inputid.drop(['genres', 'year'], axis=1)
input_id.head()

movieId                title
0           1            Toy Story
1           2              Jumanji
293       296         Pulp Fiction
1246     1274                Akira
1885     1968  Breakfast Club, The

In [22]:
inputmovies = pd.merge(input_id, userinput)
inputmovies.head()

movieId                title  rating
0        1            Toy Story     3.5
1        2              Jumanji     2.0
2      296         Pulp Fiction     5.0
3     1274                Akira     4.5
4     1968  Breakfast Club, The     5.0

**We're going to start by learning the input's preferences, so let's get the subset of movies that the input has watched from the Dataframe containing genres defined with binary values.**


In [23]:
usermovies = movies_genres[movies_genres['title'].isin(input_id['title'].to_list())]
usermovies

movieId                title  \
0           1            Toy Story   
1           2              Jumanji   
293       296         Pulp Fiction   
1246     1274                Akira   
1885     1968  Breakfast Club, The   

                                                 genres  year  Adventure  \
0     [Adventure, Animation, Children, Comedy, Fantasy]  1995        1.0   
1                        [Adventure, Children, Fantasy]  1995        1.0   
293                    [Comedy, Crime, Drama, Thriller]  1994        0.0   
1246             [Action, Adventure, Animation, Sci-Fi]  1988        1.0   
1885                                    [Comedy, Drama]  1985        0.0   

      Animation  Children  Comedy  Fantasy  Romance  ...  Horror  Mystery  \
0           1.0       1.0     1.0      1.0      0.0  ...     0.0      0.0   
1           0.0       1.0     0.0      1.0      0.0  ...     0.0      0.0   
293         0.0       0.0     1.0      0.0      0.0  ...     0.0      0.0   
1246        1.0       0.0     0.0      0.0      0.0  ...     0.0      0.0   
1885        0.0       0.0     1.0      0.0      0.0  ...     0.0      0.0   

      Sci-Fi  IMAX  Documentary  War  Musical  Western  Film-Noir  \
0        0.0   0.0          0.0  0.0      0.0      0.0        0.0   
1        0.0   0.0          0.0  0.0      0.0      0.0        0.0   
293      0.0   0.0          0.0  0.0      0.0      0.0        0.0   
1246     1.0   0.0          0.0  0.0      0.0      0.0        0.0   
1885     0.0   0.0          0.0  0.0      0.0      0.0        0.0   

      (no genres listed)  
0                    0.0  
1                    0.0  
293                  0.0  
1246                 0.0  
1885                 0.0  

[5 rows x 24 columns]

We'll only need the actual genre table, so let's clean this up a bit by resetting the index and dropping the movieId, title, genres and year columns.


In [24]:
usermovies = usermovies.reset_index(drop=True)
usermovies

movieId                title  \
0        1            Toy Story   
1        2              Jumanji   
2      296         Pulp Fiction   
3     1274                Akira   
4     1968  Breakfast Club, The   

                                              genres  year  Adventure  \
0  [Adventure, Animation, Children, Comedy, Fantasy]  1995        1.0   
1                     [Adventure, Children, Fantasy]  1995        1.0   
2                   [Comedy, Crime, Drama, Thriller]  1994        0.0   
3             [Action, Adventure, Animation, Sci-Fi]  1988        1.0   
4                                    [Comedy, Drama]  1985        0.0   

   Animation  Children  Comedy  Fantasy  Romance  ...  Horror  Mystery  \
0        1.0       1.0     1.0      1.0      0.0  ...     0.0      0.0   
1        0.0       1.0     0.0      1.0      0.0  ...     0.0      0.0   
2        0.0       0.0     1.0      0.0      0.0  ...     0.0      0.0   
3        1.0       0.0     0.0      0.0      0.0  ...     0.0      0.0   
4        0.0       0.0     1.0      0.0      0.0  ...     0.0      0.0   

   Sci-Fi  IMAX  Documentary  War  Musical  Western  Film-Noir  \
0     0.0   0.0          0.0  0.0      0.0      0.0        0.0   
1     0.0   0.0          0.0  0.0      0.0      0.0        0.0   
2     0.0   0.0          0.0  0.0      0.0      0.0        0.0   
3     1.0   0.0          0.0  0.0      0.0      0.0        0.0   
4     0.0   0.0          0.0  0.0      0.0      0.0        0.0   

   (no genres listed)  
0                 0.0  
1                 0.0  
2                 0.0  
3                 0.0  
4                 0.0  

[5 rows x 24 columns]

In [25]:
usermovies = usermovies.drop(['title', 'genres', 'year', 'movieId'], axis = 1)
usermovies

Adventure  Animation  Children  Comedy  Fantasy  Romance  Drama  Action  \
0        1.0        1.0       1.0     1.0      1.0      0.0    0.0     0.0   
1        1.0        0.0       1.0     0.0      1.0      0.0    0.0     0.0   
2        0.0        0.0       0.0     1.0      0.0      0.0    1.0     0.0   
3        1.0        1.0       0.0     0.0      0.0      0.0    0.0     1.0   
4        0.0        0.0       0.0     1.0      0.0      0.0    1.0     0.0   

   Crime  Thriller  Horror  Mystery  Sci-Fi  IMAX  Documentary  War  Musical  \
0    0.0       0.0     0.0      0.0     0.0   0.0          0.0  0.0      0.0   
1    0.0       0.0     0.0      0.0     0.0   0.0          0.0  0.0      0.0   
2    1.0       1.0     0.0      0.0     0.0   0.0          0.0  0.0      0.0   
3    0.0       0.0     0.0      0.0     1.0   0.0          0.0  0.0      0.0   
4    0.0       0.0     0.0      0.0     0.0   0.0          0.0  0.0      0.0   

   Western  Film-Noir  (no genres listed)  
0      0.0        0.0                 0.0  
1      0.0        0.0                 0.0  
2      0.0        0.0                 0.0  
3      0.0        0.0                 0.0  
4      0.0        0.0                 0.0

Now we're ready to start learning the input's preferences!

To do this, we're going to turn each genre into weights. We can do this by using the input's reviews and multiplying them into the input's genre table and then summing up the resulting table by column. This operation is actually a dot product between a matrix and a vector, so we can simply accomplish by calling the Pandas "dot" function.


In [26]:
userprofile = usermovies.transpose().dot(inputmovies['rating'])
userprofile

Adventure             10.0
Animation              8.0
Children               5.5
Comedy                13.5
Fantasy                5.5
Romance                0.0
Drama                 10.0
Action                 4.5
Crime                  5.0
Thriller               5.0
Horror                 0.0
Mystery                0.0
Sci-Fi                 4.5
IMAX                   0.0
Documentary            0.0
War                    0.0
Musical                0.0
Western                0.0
Film-Noir              0.0
(no genres listed)     0.0
dtype: float64

**Now, we have the weights for every of the user's preferences. This is known as the User Profile. Using this, we can recommend movies that satisfy the user's preferences.**

Let's start by extracting the genre table from the original dataframe:


In [27]:
genretable = movies_genres.set_index(movies_genres['movieId'])
genretable = genretable.drop(['title', 'genres', 'movieId', 'year'], axis=1)
genretable.head()

Adventure  Animation  Children  Comedy  Fantasy  Romance  Drama  \
movieId                                                                    
1              1.0        1.0       1.0     1.0      1.0      0.0    0.0   
2              1.0        0.0       1.0     0.0      1.0      0.0    0.0   
3              0.0        0.0       0.0     1.0      0.0      1.0    0.0   
4              0.0        0.0       0.0     1.0      0.0      1.0    1.0   
5              0.0        0.0       0.0     1.0      0.0      0.0    0.0   

         Action  Crime  Thriller  Horror  Mystery  Sci-Fi  IMAX  Documentary  \
movieId                                                                        
1           0.0    0.0       0.0     0.0      0.0     0.0   0.0          0.0   
2           0.0    0.0       0.0     0.0      0.0     0.0   0.0          0.0   
3           0.0    0.0       0.0     0.0      0.0     0.0   0.0          0.0   
4           0.0    0.0       0.0     0.0      0.0     0.0   0.0          0.0   
5           0.0    0.0       0.0     0.0      0.0     0.0   0.0          0.0   

         War  Musical  Western  Film-Noir  (no genres listed)  
movieId                                                        
1        0.0      0.0      0.0        0.0                 0.0  
2        0.0      0.0      0.0        0.0                 0.0  
3        0.0      0.0      0.0        0.0                 0.0  
4        0.0      0.0      0.0        0.0                 0.0  
5        0.0      0.0      0.0        0.0                 0.0

In [28]:
genretable.shape

(34208, 20)

In [29]:
recommendation = ((genretable*userprofile).sum(axis=1))/userprofile.sum()

In [30]:
recommendation.head()

movieId
1    0.594406
2    0.293706
3    0.188811
4    0.328671
5    0.188811
dtype: float64

With the input's profile and the complete list of movies and their genres in hand, we're going to take the weighted average of every movie based on the input profile and recommend the top twenty movies that most satisfy it.


In [31]:
recommendationtable = recommendation.sort_values(ascending=False)
recommendationtable.head()

movieId
5018      0.748252
26093     0.734266
27344     0.720280
148775    0.685315
6902      0.678322
dtype: float64

In [32]:
movies.loc[movies['movieId'].isin(recommendationtable.head(20).keys())]

movieId                                              title  \
664        673                                          Space Jam   
1824      1907                                              Mulan   
2902      2987                           Who Framed Roger Rabbit?   
4923      5018                                           Motorama   
6793      6902                                      Interstate 60   
8605     26093         Wonderful World of the Brothers Grimm, The   
8783     26340  Twelve Tasks of Asterix, The (Les douze travau...   
9296     27344  Revolutionary Girl Utena: Adolescence of Utena...   
9825     32031                                             Robots   
11716    51632                            Atlantis: Milo's Return   
11751    51939                TMNT (Teenage Mutant Ninja Turtles)   
13250    64645                                  The Wrecking Crew   
16055    81132                                             Rubber   
18312    91335                                      Gruffalo, The   
22778   108540           Ernest & Célestine (Ernest et Célestine)   
22881   108932                                     The Lego Movie   
25218   117646                     Dragonheart 2: A New Beginning   
26442   122787                                       The 39 Steps   
32854   146305                             Princes and Princesses   
33509   148775                Wizards of Waverly Place: The Movie   

                                                  genres  year  
664    [Adventure, Animation, Children, Comedy, Fanta...  1996  
1824   [Adventure, Animation, Children, Comedy, Drama...  1998  
2902   [Adventure, Animation, Children, Comedy, Crime...  1988  
4923   [Adventure, Comedy, Crime, Drama, Fantasy, Mys...  1991  
6793   [Adventure, Comedy, Drama, Fantasy, Mystery, S...  2002  
8605   [Adventure, Animation, Children, Comedy, Drama...  1962  
8783   [Action, Adventure, Animation, Children, Comed...  1976  
9296   [Action, Adventure, Animation, Comedy, Drama, ...  1999  
9825   [Adventure, Animation, Children, Comedy, Fanta...  2005  
11716  [Action, Adventure, Animation, Children, Comed...  2003  
11751  [Action, Adventure, Animation, Children, Comed...  2007  
13250  [Action, Adventure, Comedy, Crime, Drama, Thri...  1968  
16055  [Action, Adventure, Comedy, Crime, Drama, Film...  2010  
18312    [Adventure, Animation, Children, Comedy, Drama]  2009  
22778  [Adventure, Animation, Children, Comedy, Drama...  2012  
22881  [Action, Adventure, Animation, Children, Comed...  2014  
25218  [Action, Adventure, Comedy, Drama, Fantasy, Th...  2000  
26442  [Action, Adventure, Comedy, Crime, Drama, Thri...  1959  
32854  [Animation, Children, Comedy, Drama, Fantasy, ...  2000  
33509  [Adventure, Children, Comedy, Drama, Fantasy, ...  2009